Deep Learning CNN model to recognize X-Ray
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''
   
Data Source: Kaggle

'''########################## IMAGE PRE-PROCESSING for TRAINING and TESTING data ##############################'''

In [1]:
#Image path for Training and Testing
Trainingimagepath='D:\\DS\\Python\\CNN assignment\\chest_xray\\train'
Testingimagepath='D:\\DS\\Python\\CNN assignment\\chest_xray\\train'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
# Defining pre-processing transformations on raw images of training data
Train_datagen=ImageDataGenerator(rescale=1./255, zoom_range=0.1,
                                 horizontal_flip=True,
                                 shear_range=0.1
                                )
# Defining pre-processing transformations on raw images of testing data
Test_datagen=ImageDataGenerator(rescale=1./255, zoom_range=0.1,
                                horizontal_flip=True,
                                 shear_range=0.1
                                )
# Generating the Training Data
Training_set=Train_datagen.flow_from_directory(Trainingimagepath, target_size=(64,64),
                                              batch_size=100, class_mode='categorical')
# Generating the Testing Data
Test_set=Train_datagen.flow_from_directory(Testingimagepath, target_size=(64,64),
                                              batch_size=100, class_mode='categorical')
# Printing class labels for each face
Test_set.class_indices

Found 5216 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


{'NORMAL': 0, 'PNEUMONIA': 1}

In [3]:
'''#################### Creating lookup table for all faces ##############################'''
# class_indices have the numeric tag for each face

Trainclasses=Training_set.class_indices
# Storing the face and the numeric tag for future reference
Resultmap1={}
for facevalue, facename in zip(Trainclasses.values(), Trainclasses.keys()):
    Resultmap1[facevalue]=facename
# Saving the face map for future reference    
import pickle
with open('D:\\DS\\Python\\CNN assignment\\chest_xray\\Resultmap1.pkl','wb') as f:
    pickle.dump(Resultmap1, f, pickle.HIGHEST_PROTOCOL)
    
print('Mapping X-ray and its Type:', Resultmap1)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(Resultmap1)
print('No of neurons', OutputNeurons)

Mapping X-ray and its Type: {0: 'NORMAL', 1: 'PNEUMONIA'}
No of neurons 2


CNN model Creation

In [4]:
'''######################## Create CNN deep learning model ####################################'''
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPool2D, Flatten, Dense
'''Initializing the Convolutional Neural Network'''
classifier=Sequential()
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels'''
classifier.add(Convolution2D(32, kernel_size=(5,5), 
                    input_shape=(64,64,3), strides=(1,1), activation='relu'))
'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))
'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5,5), 
                     strides=(1,1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
'''# STEP--3 FLattening'''
classifier.add(Flatten())
'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64,activation='relu'))
classifier.add(Dense(OutputNeurons,activation='softmax'))
'''# Compiling the CNN'''
classifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Fitting the model

In [5]:
# Measuring the time taken by the model to train
import time
Start_time=time.time()
classifier.fit_generator(Training_set, steps_per_epoch=30, epochs=10,
                        validation_data=Test_set, validation_steps=10)


End_time=time.time()

print('Time Taken',  round((End_time-Start_time)/60))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
30/30 [==============================] - 140s 5s/step - loss: 0.5590 - accuracy: 0.7553 - val_loss: 0.3666 - val_accuracy: 0.8280
Epoch 2/10
30/30 [==============================] - 117s 4s/step - loss: 0.2747 - accuracy: 0.8840 - val_loss: 0.2609 - val_accuracy: 0.8930
Epoch 3/10
30/30 [==============================] - 99s 3s/step - loss: 0.2071 - accuracy: 0.9180 - val_loss: 0.2279 - val_accuracy: 0.9020
Epoch 4/10
30/30 [==============================] - 90s 3s/step - loss: 0.2057 - accuracy: 0.9133 - val_loss: 0.1631 - val_accuracy: 0.9320
Epoch 5/10
30/30 [==============================] - 87s 3s/step - loss: 0.1854 - accuracy: 0.9243 - val_loss: 0.2034 - val_accuracy: 0.9060
Epoch 6/10
30/30 [==============================] - 86s 3s/step - loss: 0.1611 - accuracy: 0.9324 - val_loss: 0.1494 - val_accuracy: 0.9380
Epoch 7/10
30/30 [==============================] - 84s 3s/step - loss: 0.1686 - a

Saving the model

In [6]:
classifier.save("D:\\DS\\Python\\CNN assignment\\chest_xray\\classifier1.pkl")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: D:\DS\Python\CNN assignment\chest_xray\classifier1.pkl\assets


Testing the model on a different face

In [7]:
'''########################## Making single predictions ############################'''
import numpy as np
from keras.preprocessing import image

test_image='D:\\DS\\Python\\CNN assignment\\chest_xray\\val\\PNEUMONIA\\person1946_bacteria_4874.jpeg'
TestImage=image.load_img(test_image,target_size=(64,64))
TestImage=image.img_to_array(TestImage)

TestImage=np.expand_dims(TestImage, axis=0)

result=classifier.predict(TestImage,verbose=0)

print('####'*10)
print('Prediction is: ',Resultmap1[np.argmax(result)])

########################################
Prediction is:  PNEUMONIA


The Model has accurately predicted with the image of X-Ray whether the person has Pneumonia or Not.